
# TODO

~~read test files~~

do normalization separately for fbank and mfcc?

~~padding~~

~~Use 48 for training and map after prediction~~

outliers?

adam?

use custom metric (edit distance)

~~BLSTM~~

~~monitor for modelcheckpoint and earlystopping~~

~~no validation?~~

Add padding as 40th class?

~~dropout / recurrent dropout~~

move dropout into LSTM

RNN-CNN

ensemble

~~ReLU inside LSTM~~

better squeezing method

batch normalization?

**observe data (esp. labels)**

In [ ]:
import numpy as np
np.random.seed(666)
from utils import *

# # GPU usage
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.6
# set_session(tf.Session(config=config))

In [ ]:
sentence2frames = dict()
read_ark(DATA_PATH + 'fbank/train.ark', sentence2frames)
read_ark(DATA_PATH + 'mfcc/train.ark', sentence2frames)

# number of training sentences
NUM_SENTENCES = len(sentence2frames)
print(NUM_SENTENCES)

# number of frames in this sentence
print(len(sentence2frames['faem0_si1392']))

# number of fbank+mfcc features in a frame
print(len(sentence2frames['faem0_si1392'][0]))

# first feature
print(sentence2frames['faem0_si1392'][0][0])

In [ ]:
sentence2labels = dict()
read_label(DATA_PATH + 'label/train.lab', sentence2labels)

# number of training sentences
print(len(sentence2labels))

# number of frames in this sentence
print(len(sentence2labels['faem0_si1392']))

# first label
print(sentence2labels['faem0_si1392'][0])

In [ ]:
sentences = sorted(sentence2labels)
X = np.zeros((NUM_SENTENCES, MAX_SENTENCE_LEN, FEATURE_LEN))
y = np.zeros((NUM_SENTENCES, MAX_SENTENCE_LEN, NUM_CLASSES))

for i, s in enumerate(sentences):
    frames = sentence2frames[s]
    X[i, :len(frames)] = frames
    y[i, :len(frames)] = np.stack([one_hot(NUM_CLASSES, l) for l in sentence2labels[s]])

print(X.shape)
print(y.shape)

In [ ]:
model = build_model(summary=True)

In [ ]:
monitor = 'loss'
model_filepath = MODEL_PATH + 'model_{monitor}'.format(monitor=monitor) + '_{epoch:03d}_{loss:.4f}_mapafter_recurrent_dropout_0.1_2lstm.h5'
checkpoint = ModelCheckpoint(model_filepath, monitor=monitor, verbose=0, save_best_only=True)
earlystop = EarlyStopping(monitor=monitor, patience=2, verbose=0)
model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[checkpoint])

In [ ]:
'''
monitor = val_acc
Epoch 36/200
3326/3326 [==============================] - 77s - loss: 0.3140 - acc: 0.2903 - val_loss: 0.3337 - val_acc: 0.2845

monitor = val_loss
Epoch 23/500
3326/3326 [==============================] - 172s - loss: 0.3495 - acc: 0.2870 - val_loss: 0.3622 - val_acc: 0.2925

monitor = acc (no validation)
Epoch 133/500
3696/3696 [==============================] - 109s - loss: 0.2369 - acc: 0.3140
'''